<p>Identify most popular free apps categories</p>
<p><b>Aims:</b></p>
<li>Free</li>
<li>User rating</li>
<li>Size</li>
<li>Genre</li>

In [376]:
from csv import reader
import re
import operator
opened_file_1 = open('AppleStore.csv', encoding="utf8")
opened_file_2 = open('googleplaystore.csv', encoding="utf8")
ios_data = list(reader(opened_file_1))
google_data = list(reader(opened_file_2))

In [377]:
def explore_data(dataset, start = 0, end = 1, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [378]:
print("Ios Categoies: ")
explore_data(ios_data, rows_and_columns = True)
print('\n')
print("Google Categories: ")
explore_data(google_data, rows_and_columns = True)

Ios Categoies: 
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Number of rows: 7198
Number of columns: 16


Google Categories: 
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Number of rows: 10842
Number of columns: 13


<p>Some data in ios_data and google_data are <b>duplicates!</b></p>
<p>They will be removed and only the entry with the highest number of reviews will be held as it is the newest.</p>
<p>A function finds duplicates is shown below</p>

In [379]:
#find duplicates
#index is name
def unique(apps,index):
    duplicate_apps = []
    unique_apps = []

    for app in apps:
        name = app[index]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    print("This is an example of duplicates apps!")
    for app in apps:
        if app[0] == duplicate_apps[0]:
            print(app)
    return unique_apps

In [380]:
print("Criteria for duplicates: ")
print("\t The entry with greatest number of reviews is assumed to be valid, as it is the lates.")

Criteria for duplicates: 
	 The entry with greatest number of reviews is assumed to be valid, as it is the lates.


<p>The functions below help to create a list of the apps of interest</p>
<p>
<b><u>Requirements:</u></b>
<li>Free</li>
<li>Remove duplicates</li>
<li>Remove apps with insignificant number of reviews</li>
</p>
<p>PS:</p>
<p>When passing the "apps" parameter check if the list has a header</p>

In [381]:
#converts string numbers to float
#index, reviews
def st_to_fl(apps,index):
    for app in apps:
        snum = app[index]
        ln = len(snum)#lenght of num string
        if snum[-1] == 'M':
            num = snum[:ln-2]
            app[index] = float(num)*1000000
        else:
            app[index] = float(snum)
    return apps

In [382]:
#remove duplicates
#keep english apps
#index_name, name
#index_rev, reviews
def duplicates(apps,index_name,index_rev,unique):
    rev_max = {}
    for app in unique:
        rev_max[app] = 0
    for app in apps:
        num = app[index_rev]
        if num > rev_max[app[index_name]]:
            rev_max[app[index_name]] = num #Creates dictionary with unique names and corresponding max_review number
    print('\n')
    return rev_max


In [383]:
#create list from dictionary
#index_name, name
#index_rev, review
def dic_to_li(dic,apps,index_name,index_rev):
    new_li = []
    for app in apps:
        if (app[index_name] in dic) and (app[index_rev] == dic[app[index_name]]):
            new_li.append(app)
    return new_li

In [384]:
#remove apps with N reviews or less as is not a good sample
#index, review
#N, least number of reviews
def valid_apps(apps,index,N):
    i = 0
    for app in apps:
        if app[index] < N:
            del apps[i]
        i += 1
    return apps

In [385]:
#keep free apps
#index is price
def free(apps,index):
    count = 0
    for app in apps:
        num = re.findall(r"[-+]?\d*\.\d+|\d+", app[index])
        if (len(num) != 0) and (float(num[0]) != 0):
            del apps[count]
        count += 1
    return apps

In [386]:
print(len(google_data))
google_data_2 = st_to_fl(google_data[1:],3)
google_data_3 = duplicates(google_data_2,0,3,unique(google_data,0))
clean_google = dic_to_li(google_data_3,google_data_2,0,3)
google_data_4 = valid_apps(clean_google,3,10)
google_data_5 = free(google_data_4,7)
print(len(google_data_5))
print('\n')

10842
This is an example of duplicates apps!
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', 80805.0, 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', 80805.0, 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', 80804.0, 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']


8260




In [387]:
print(len(ios_data))
ios_data_2 = st_to_fl(ios_data[1:],5)
ios_data_3 = duplicates(ios_data_2,1,5,unique(ios_data,1))
clean_ios = dic_to_li(ios_data_3,ios_data_2,1,5)
ios_data_4 = valid_apps(clean_ios,5,10)
ios_data_5 = free(ios_data_4,4)
print(len(ios_data_5))

7198
This is an example of duplicates apps!


4554


In [388]:
#check for english characters
#index, name
def english(apps,index):
    for app in apps:
        count = 0
        for ch in app[index]:
            if ord(ch) > 127:
                count += 1
        if count > 3:
            apps.remove(app)
    return apps

In [389]:
new_google = english(google_data_5,0)
print(new_google[0])
new_ios= english(ios_data_5,1)
print(new_ios[0])

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', 159.0, '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['284882215', 'Facebook', '389879808', 'USD', '0.0', 2974676.0, '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


In [390]:
#finds frequency of genre
#calculates average rating for each genre
#index, genre
#returns percentage of each genre
def genre_freq(apps,index_gen,index_rat):
    genre = {}
    total = 0
    for app in apps:
        if app[index_gen] in genre: 
            rating = float(app[index_rat])
            genre[app[index_gen]][0] += 1
            genre[app[index_gen]][1] += rating
        else:
            rating = float(app[index_rat])
            genre[app[index_gen]] = [1,rating]
    for key in genre.keys():
        freq = genre[key][0]
        percentage = (genre[key][0]/len(apps)) * 100
        avg_rating = genre[key][1]/genre[key][0]
        genre[key] = (freq,percentage,avg_rating)
    return genre

<p>Finding the genre and category frequency of Google Data</p>
<p>Finding the prime genre frequency of IOS Data</p>
<p><b>Prints: (Genre,(frequncy, percentage))</b></p>

In [391]:
google_categ = genre_freq(new_google,1,2)
google_categ = sorted(google_categ.items(),key=operator.itemgetter(1),reverse = True)
print("Google Categories: ")
print(google_categ)
google_genre = genre_freq(new_google,9,2)
google_genre = sorted(google_genre.items(),key=operator.itemgetter(1),reverse = True)
print("Google Genre: ")
print(google_genre)
ios_genre = genre_freq(new_ios,11,7)
ios_genre = sorted(ios_genre.items(),key=operator.itemgetter(1),reverse = True)
print("IOS/Prime Genre: ")
print(ios_genre)
new_google.insert(0,google_data[0])
new_ios.insert(0,ios_data[0])

Google Categories: 
[('FAMILY', (1538, 18.696814976902505, nan)), ('GAME', (858, 10.430342815463165, nan)), ('TOOLS', (673, 8.181376124483347, nan)), ('PRODUCTIVITY', (307, 3.732069049355701, nan)), ('BUSINESS', (302, 3.6712861658157063, nan)), ('FINANCE', (296, 3.598346705567712, nan)), ('MEDICAL', (296, 3.598346705567712, nan)), ('LIFESTYLE', (294, 3.574033552151714, nan)), ('SPORTS', (278, 3.3795283248237293, nan)), ('HEALTH_AND_FITNESS', (264, 3.209336250911744, nan)), ('PERSONALIZATION', (261, 3.1728665207877462, nan)), ('COMMUNICATION', (251, 3.0513007537077557, nan)), ('PHOTOGRAPHY', (250, 3.039144176999757, nan)), ('NEWS_AND_MAGAZINES', (231, 2.8081692195477754, nan)), ('SOCIAL', (219, 2.662290299051787, nan)), ('SHOPPING', (212, 2.5771942620957935, nan)), ('TRAVEL_AND_LOCAL', (204, 2.4799416484318018, nan)), ('BOOKS_AND_REFERENCE', (169, 2.054461463651836, nan)), ('DATING', (166, 2.017991733527839, nan)), ('VIDEO_PLAYERS', (153, 1.8599562363238513, nan)), ('EDUCATION', (123, 1